<a href="https://colab.research.google.com/github/farhanaz-08/ML_Lab/blob/main/Fake_News_Detection_Restnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone Ashia's repo

In [3]:

!git clone https://github.com/ashia-002/Fake-News-Detection-Bilingual-Multimodel.git


Cloning into 'Fake-News-Detection-Bilingual-Multimodel'...
remote: Enumerating objects: 10046, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 10046 (delta 1), reused 1 (delta 0), pack-reused 10041 (from 3)
Receiving objects: 100% (10046/10046), 1.72 GiB | 16.40 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (13988/13988), done.


Confirm repo is cloned

In [3]:
!ls

Fake-News-Detection-Bilingual-Multimodel  sample_data


checking dataset location inside the repo

In [4]:
!ls Fake-News-Detection-Bilingual-Multimodel


 AI_Lab01.ipynb   file				   logs        README.md
 CNN_Pipeline	 'image Clasification CNN.ipynb'   models      src
 data		  LargeDataset			   notebooks


Selecting required dataset path

In [5]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# adjust this path based on actual structure
dataset_path = "/content/Fake-News-Detection-Bilingual-Multimodel/data"

full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
print("Total images:", len(full_dataset))
print("Classes:", full_dataset.classes)


Total images: 4000
Classes: ['fake', 'real']


Splitting the Dataset
70% training
20% validation
10% testing

In [13]:
from torch.utils.data import random_split, DataLoader

# Define split sizes
train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✅ Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")


NameError: name 'full_dataset' is not defined

Importing Libraries

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models


Loading Pre-trained RestNet-50

In [6]:
resnet50 = models.resnet50(weights='IMAGENET1K_V1')


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 220MB/s]


Freezing early layers

In [7]:
for param in resnet50.parameters():
    param.requires_grad = False


Changing classification layer to 2

In [8]:
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 2),
    nn.LogSoftmax(dim=1)
)


Defining loss & optimizer

In [9]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(resnet50.fc.parameters(), lr=0.0001)


Moving model to GPU

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50.to(device)
print("Training on:", device)


Training on: cuda


Training and validating the model

In [12]:
from tqdm import tqdm

epochs = 10

for epoch in range(epochs):
    resnet50.train()
    running_loss = 0.0

    # 🔹 Training with progress bar
    train_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}] Training", leave=False)
    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Update progress bar with current loss
        train_bar.set_postfix(loss=loss.item())

    # 🔹 Validation with progress bar
    resnet50.eval()
    val_loss, val_correct = 0.0, 0
    val_bar = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{epochs}] Validation", leave=False)

    with torch.no_grad():
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = resnet50(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            val_correct += (preds == labels).sum().item()

            # Update validation bar
            val_bar.set_postfix(val_loss=loss.item())

    # 🔹 Summary per epoch
    print(f"✅ Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {running_loss/len(train_loader):.4f} "
          f"Val Loss: {val_loss/len(val_loader):.4f} "
          f"Val Acc: {val_correct/len(val_dataset):.4f}")


NameError: name 'train_loader' is not defined